In [61]:
# A program that looks at any webpage on the internet, 
# scrape the content of the webpage and summarize it and 
# present a short summary of the webpage

In [62]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup # for web scraping (web parsing)
from openai import OpenAI
from IPython.display import Markdown, display

In [63]:
# Load environment variables in a file .env

load_dotenv()
openai_key=os.getenv("OPENAI_API_KEY")

# Check the key
if not openai_key:
    print("An API key was not found")
elif openai_key[:8]!="sk-proj-":
    print("An API key was found, but it dosen't start with sk-proj-")
elif openai_key.strip()!= openai_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them")
else:
    print("An API key was found and looks good")

An API key was found and looks good


In [64]:
# Create an instance of openai

openai = OpenAI()

In [65]:
# A class to represent a Webpage

class Website:
    """
    A utility calss to represent a website that we have scraped
    """
    url: str
    title: str
    text: str

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeutifulSoup library
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [66]:
# output

site = Website("https://ihvnigeria.org")
print(site.title)
print(site.text)

Institute of Human Virology, Nigeria. - IHV NIGERIA.org
+234 9461 0342
[email protected]
Working Hours - Mon - Thu: 8:00 - 17:00; Fri: 8:00 - 14:00
What are you looking for?
Institute of Human Virology, Nigeria - IHV NIGERIA
Contact Us
Toggle menu
Home
About Us
Our History
Leadership & Team
Board of Directors
Executive Team
Directors, Deputies & Associates
Regional & State Managers, Technical Directors
Unit Heads
Technical Advisers
Corporate Activities
Office of the CEO
Global Fund Programs
Program Communication and Treatment Awareness Unit
PEPFAR Coordinating Unit
Learning and Development Unit
Office of The COO
Institutional Policies
Administration Department
Information Technology Unit
Maintenance Unit
Human Resources Unit
Office Management Unit
Travels Unit
Building Services Unit
Safety and Security Unit
Finance Department
Financial Services
Financial Reporting, Budgets & Management Information System (MIS)
Grants Management Department
Supply Chain Management Department
Board Secret

In [67]:
# Define our system prompt

system_prompt = "You are an assistant that analyzes the contents of a website" \
"and provides a short summary, ignoring text that might be navigation related." \
"Respond in markdown."

In [68]:
# A function that writes a User Prompt that asks for summaries of websites

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt +="\nThe contents of this website is as follows:"
    "please provide a short summary of this website in markdown."
    "If it includes news or announcement, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [69]:
print(user_prompt_for(site))

You are looking at a website titled Institute of Human Virology, Nigeria. - IHV NIGERIA.org
The contents of this website is as follows:+234 9461 0342
[email protected]
Working Hours - Mon - Thu: 8:00 - 17:00; Fri: 8:00 - 14:00
What are you looking for?
Institute of Human Virology, Nigeria - IHV NIGERIA
Contact Us
Toggle menu
Home
About Us
Our History
Leadership & Team
Board of Directors
Executive Team
Directors, Deputies & Associates
Regional & State Managers, Technical Directors
Unit Heads
Technical Advisers
Corporate Activities
Office of the CEO
Global Fund Programs
Program Communication and Treatment Awareness Unit
PEPFAR Coordinating Unit
Learning and Development Unit
Office of The COO
Institutional Policies
Administration Department
Information Technology Unit
Maintenance Unit
Human Resources Unit
Office Management Unit
Travels Unit
Building Services Unit
Safety and Security Unit
Finance Department
Financial Services
Financial Reporting, Budgets & Management Information System (MI

In [70]:
def messages_for(website):
    return [
        {"role":"system", "content": system_prompt},
        {"role":"user", "content":user_prompt_for(website)}
    ]

In [71]:
# messages_for(site)

In [72]:
# Using OpenAI API to summarize

def summarize(url):
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages_for(url),
        stream=True
    )
    
  # Accumulate content as it streams
    full_content = ""
    output = display("", display_id=True)  # Create a display object that can be updated
    
    for chunk in response:
        if chunk.choices and chunk.choices[0].delta.content:
            content = chunk.choices[0].delta.content
            full_content += content
            
            # Update the display with the current markdown content
            output.update(Markdown(full_content))
    
    return full_content

In [73]:
# summarize(site)

In [74]:
# A function to display this nicely in the jupyter output using markdown

def display_summary(url):
    summarize(url)
    

In [75]:
display_summary(site)

# Summary of the Institute of Human Virology, Nigeria (IHV NIGERIA)

The Institute of Human Virology, Nigeria (IHVN) focuses on improving health outcomes in Nigeria through various programs and initiatives. The institute's mission is to serve as a center of excellence in health service implementation, capacity building, and research. It aims to ensure equitable access to healthcare for individuals and communities using innovative and evidence-based strategies.

## Key Focus Areas
- **HIV Treatment and Prevention**: A major emphasis on programs for adults, children, adolescents, and pregnant women.
- **TB and Malaria Services**: Initiatives targeting the prevention and treatment of TB and malaria.
- **Research**: A commitment to ethical, collaborative research, especially through the International Research Center of Excellence (IRCE).

## Core Values
IHVN operates on principles of:
- Integrity
- Respect for individuals' dignity
- Innovation
- Transparency
- Accountability
- Excellence
- Teamwork

## Recent Highlights
- IHVN is actively involved in a variety of ongoing projects, including initiatives funded by PEPFAR and the Global Fund aimed at improving healthcare delivery.
- The institute recently launched programs supporting food security and prevention initiatives in vulnerable communities.
- Trainings and conferences, such as the Public Health Data Analysis Course and EXCEL-RITE Conference, emphasize the importance of research integrity and capacity building.

The organization plays a pivotal role in responding to public health challenges in Nigeria by providing training, implementing health programs, and conducting research tailored to the needs of the local population.